In [1]:
import tensorflow as tf
import pandas as pd
import pyarrow as pa
import pyarrow.parquet as pq
from tensorflow.keras.layers import Dense, Flatten, Conv2D, Input
from tensorflow.keras import Model
from tensorflow.keras.experimental import LinearModel

In [2]:
%config Completer.use_jedi = False

In [3]:
# Uncomment the following to disable GPU
tf.config.set_visible_devices([], 'GPU')
print(tf.config.get_visible_devices())
gpus = tf.config.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


InternalError: CUDA runtime implicit initialization on GPU:0 failed. Status: out of memory

# Import parquet data

In [ ]:
ds = pq.ParquetDataset("/var/log/fancontrol/featurelog")
ptable = ds.read().to_pandas()

In [ ]:
# Get rid of outliers in power feture
def clamp(num, min_value, max_value):
   return max(min(num, max_value), min_value)
power = ptable['power'].apply(lambda a: clamp(a[0], 0.0, 99.0))

In [ ]:
def feature_names():
    l = list(ptable.columns.values)
    l.remove('label')
    return l

def generator_types():
    return ({k: tf.float32 for k in feature_names()}, tf.float32)

def dim(val):
    return 1 if isinstance(val, float) or isinstance(val, int) else len(val)

def generator_shapes():
    _, row = next(ptable.iterrows())
    shapes = {k: (1, dim(row[k])) for k in feature_names()}
    return tuple([shapes, (1,)])

print(feature_names())
print(generator_types())
print(generator_shapes())

In [ ]:
def convert(key, value):
    if isinstance(value, float) or isinstance(value, int):
        return tf.reshape(tf.convert_to_tensor(value, tf.float32, name=key), (1, 1))
    return tf.reshape(tf.convert_to_tensor(value, tf.float32, name=key), (1, dim(value)))

def generator():
    for index, row in ptable.iterrows():
        f = {k: convert(k, row[k]) for k in feature_names()}
        l = tf.convert_to_tensor(list([float(row['label'])]), tf.float32, name='label')
        yield f, l

In [ ]:
all = tf.data.Dataset.from_generator(generator, output_types=generator_types(), output_shapes=generator_shapes()) \
    .shuffle(10, reshuffle_each_iteration=False)
n = next(all.batch(2).as_numpy_iterator())
print(n)

In [ ]:
tf.shape(n[0]['temp']).numpy()

In [ ]:
tf.shape(n[1]).numpy()

In [ ]:
def is_test(x, y):
    return x % 4 == 0
def is_train(x, y):
    return not is_test(x, y)
recover = lambda x,y: y
test_dataset = all.enumerate() \
                    .filter(is_test) \
                    .map(recover)

train_dataset = all.enumerate() \
                    .filter(is_train) \
                    .map(recover)

# Lets build a simple model

In [ ]:
from tensorflow.keras import layers
from tensorflow import keras

In [ ]:
temp_input = Input(shape=(10,), name="temp")
cpu_idle_input = Input(shape=(8,), name="cpu_idle")
power_input = Input(shape=(1,), name="power")
fan_input = Input(shape=(1,), name="fan_rpm")
all_input = [power_input, temp_input, cpu_idle_input, fan_input]

In [ ]:
x = layers.concatenate(all_input)

In [ ]:
fanlevel_output = Dense(1, name="level")(x)

In [ ]:
linear_model=keras.Model(inputs=all_input, outputs=[fanlevel_output])
linear_model.compile(optimizer='adam', loss='msle')

In [ ]:
keras.utils.plot_model(linear_model, "multi_input_and_output_model.png", show_shapes=True)

In [ ]:
linear_model.fit(train_dataset, epochs=1, batch_size=1024)

In [ ]:
linear_model.save("first_train")